<a href="https://colab.research.google.com/github/nishitjain97/NLP_538_Fall_2022_Project_HaND/blob/main/BERT_Predictions_and_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook for BERT predictions and evaluation

## Package Loading

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [17]:
!pip install tensorflow
!pip install transformers
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [122]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import random
import torch
import tensorflow as tf

from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import multilabel_confusion_matrix, f1_score, precision_score, recall_score

## Utilities

In [57]:
def encode_sentiment(sentiment):
  if sentiment == 'Negative':
    return 0
  elif sentiment == 'Neutral':
    return 1
  else:
    return 2

In [58]:
def process_data(data, tokenizer):
  input_ids = {}
  attention_masks = {}
  labels = {}
  
  for key in data.keys():
    print(key)
    input_ids[key] = []
    attention_masks[key] = []
    
    for sentence in data[key]['MASKED_DOCUMENT']:
      encoded_dict = tokenizer.encode_plus(
                          sentence,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = MAX_LENGTH,           # Pad & truncate all sentences.
                          padding = 'max_length',
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                          truncation=True
                    )
      input_ids[key].append(encoded_dict['input_ids'])
      attention_masks[key].append(encoded_dict['attention_mask'])

    input_ids[key] = torch.cat(input_ids[key], dim=0)
    attention_masks[key] = torch.cat(attention_masks[key], dim=0)

  for key in data.keys():
    labels[key] = data[key]['TRUE_SENTIMENT'].apply(lambda x: encode_sentiment(x))

  dataset = {}
  
  for key in data.keys():
    dataset[key] = TensorDataset(input_ids[key], attention_masks[key], torch.tensor(labels[key]))

  return dataset

In [59]:
def generate_dataloaders(config, dataset):
  dataloaders = {}
  
  for key in dataset.keys():
    dataloaders[key] = DataLoader(
        dataset[key],
        sampler = RandomSampler(dataset[key]),
        batch_size = config[key]
    )

  return dataloaders

In [99]:
# Function to calculate the accuracy of our predictions vs labels
def get_accuracy(preds, labels):
  return np.sum(preds == labels) / len(labels)

def multiclass_confusion_matrix(preds, labels):
  return multilabel_confusion_matrix(labels, preds)

In [124]:
def make_predictions(model, dataloader):
  model.eval()

  # Tracking variables 
  predictions , true_labels = [], []

  # Predict 
  for batch in dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions.
        result = model(b_input_ids, 
                      token_type_ids=None, 
                      attention_mask=b_input_mask,
                      return_dict=True)

    logits = result.logits

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)
    
  return predictions, true_labels

def get_precision(cm):
  true_pos = np.diag(cm) 
  return np.sum(true_pos / np.sum(cm, axis=0))

def get_recall(cm):
  true_pos = np.diag(cm)
  return np.sum(true_pos / np.sum(cm, axis=1))

def evaluate_predictions(predictions, true_labels):
  predictions = np.argmax(predictions, axis=1).flatten()
  true_labels = true_labels.flatten()
  
  acc = get_accuracy(predictions, true_labels)
  p = precision_score(true_labels, predictions, average=None)
  r = recall_score(true_labels, predictions, average=None)
  f1 = f1_score(true_labels, predictions, average=None)
  macro_f1 = f1_score(true_labels, predictions, average='macro')
  
  return acc, p, r, f1, macro_f1

## Configurations

In [62]:
MAX_LENGTH = 512

batch_sizes = {
    'train': 8,
    'dev': 4,
    'random_test': 4,
    'fixed_test': 4
}

In [63]:
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))

In [64]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


## Load Data

In [65]:
data_path = "/content/gdrive/MyDrive/NLP_Project/"
data = {}

for item in ['random_test', 'fixed_test']:
  data[item] = pd.read_csv(os.path.join(data_path, item + '.txt'))

## Model Loading

In [66]:
tokenizer = BertTokenizer.from_pretrained("/content/gdrive/MyDrive/NLP_Project/model_save")
model = BertForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/NLP_Project/model_save")

if device.type != 'cpu':
  model.cuda()

In [67]:
dataset = process_data(data, tokenizer)

random_test
fixed_test


In [68]:
dataloaders = generate_dataloaders(batch_sizes, dataset)

In [125]:
for key, dataloader in dataloaders.items():
  predictions, true_labels = make_predictions(model, dataloader)
  predictions = np.concatenate(predictions)
  true_labels = np.concatenate(true_labels)
  acc, p, r, f1, macro_f1 = evaluate_predictions(predictions, true_labels)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
